In [1]:
def get_usb_iphone():
    Type = 'usb'
    if int(frida.__version__.split('.')[0]) < 12:
        Type = 'tether'
    device_manager = frida.get_device_manager()
    changed = threading.Event()

    def on_changed():
        changed.set()

    device_manager.on('changed', on_changed)

    device = None
    while device is None:
        devices = [dev for dev in device_manager.enumerate_devices() if dev.type == Type]
        if len(devices) == 0:
            print('Waiting for USB device...')
            changed.wait()
        else:
            device = devices[0]

    device_manager.off('changed', on_changed)

    return device


def generate_ipa(path, display_name):
    ipa_filename = display_name + '.ipa'

    print('Generating "{}"'.format(ipa_filename))
    try:
        app_name = file_dict['app']

        for key, value in file_dict.items():
            from_dir = os.path.join(path, key)
            to_dir = os.path.join(path, app_name, value)
            if key != 'app':
                shutil.move(from_dir, to_dir)

        target_dir = './' + PAYLOAD_DIR
        zip_args = ('zip', '-qr', os.path.join(os.getcwd(),"ipas", ipa_filename), target_dir)
        subprocess.check_call(zip_args, cwd=TEMP_DIR)
        shutil.rmtree(PAYLOAD_PATH)
    except Exception as e:
        print(e)
        finished.set()

def on_message(message, data):
    t = tqdm(unit='B',unit_scale=True,unit_divisor=1024,miniters=1)
    last_sent = [0]

    def progress(filename, size, sent):
        baseName = os.path.basename(filename)
        if IS_PY2 or isinstance(baseName, bytes):
            t.desc = baseName.decode("utf-8")
        else:
            t.desc = baseName
        t.total = size
        t.update(sent - last_sent[0])
        last_sent[0] = 0 if size == sent else sent

    if 'payload' in message:
        payload = message['payload']
        if 'dump' in payload:
            origin_path = payload['path']
            dump_path = payload['dump']

            scp_from = dump_path
            scp_to = PAYLOAD_PATH + '/'

            with SCPClient(ssh.get_transport(), progress = progress, socket_timeout = 60) as scp:
                scp.get(scp_from, scp_to)

            chmod_dir = os.path.join(PAYLOAD_PATH, os.path.basename(dump_path))
            chmod_args = ('chmod', '655', chmod_dir)
            try:
                subprocess.check_call(chmod_args)
            except subprocess.CalledProcessError as err:
                print(err)

            index = origin_path.find('.app/')
            file_dict[os.path.basename(dump_path)] = origin_path[index + 5:]

        if 'app' in payload:
            app_path = payload['app']

            scp_from = app_path
            scp_to = PAYLOAD_PATH + '/'
            with SCPClient(ssh.get_transport(), progress = progress, socket_timeout = 60) as scp:
                scp.get(scp_from, scp_to, recursive=True)

            chmod_dir = os.path.join(PAYLOAD_PATH, os.path.basename(app_path))
            chmod_args = ('chmod', '755', chmod_dir)
            try:
                subprocess.check_call(chmod_args)
            except subprocess.CalledProcessError as err:
                print(err)

            file_dict['app'] = os.path.basename(app_path)

        if 'done' in payload:
            finished.set()
    t.close()

def compare_applications(a, b):
    a_is_running = a.pid != 0
    b_is_running = b.pid != 0
    if a_is_running == b_is_running:
        if a.name > b.name:
            return 1
        elif a.name < b.name:
            return -1
        else:
            return 0
    elif a_is_running:
        return -1
    else:
        return 1


def cmp_to_key(mycmp):
    """Convert a cmp= function into a key= function"""

    class K:
        def __init__(self, obj):
            self.obj = obj

        def __lt__(self, other):
            return mycmp(self.obj, other.obj) < 0

        def __gt__(self, other):
            return mycmp(self.obj, other.obj) > 0

        def __eq__(self, other):
            return mycmp(self.obj, other.obj) == 0

        def __le__(self, other):
            return mycmp(self.obj, other.obj) <= 0

        def __ge__(self, other):
            return mycmp(self.obj, other.obj) >= 0

        def __ne__(self, other):
            return mycmp(self.obj, other.obj) != 0

    return K


def get_applications(device):
    try:
        applications = device.enumerate_applications()
    except Exception as e:
        sys.exit('Failed to enumerate applications: %s' % e)

    return applications


def list_applications(device):
    applications = get_applications(device)

    if len(applications) > 0:
        pid_column_width = max(map(lambda app: len('{}'.format(app.pid)), applications))
        name_column_width = max(map(lambda app: len(app.name), applications))
        identifier_column_width = max(map(lambda app: len(app.identifier), applications))
    else:
        pid_column_width = 0
        name_column_width = 0
        identifier_column_width = 0

    header_format = '%' + str(pid_column_width) + 's  ' + '%-' + str(name_column_width) + 's  ' + '%-' + str(
        identifier_column_width) + 's'
    print(header_format % ('PID', 'Name', 'Identifier'))
    print('%s  %s  %s' % (pid_column_width * '-', name_column_width * '-', identifier_column_width * '-'))
    line_format = '%' + str(pid_column_width) + 's  ' + '%-' + str(name_column_width) + 's  ' + '%-' + str(
        identifier_column_width) + 's'
    for application in sorted(applications, key=cmp_to_key(compare_applications)):
        if application.pid == 0:
            print(line_format % ('-', application.name, application.identifier))
        else:
            print(line_format % (application.pid, application.name, application.identifier))

def get_application_IDs():
    applications = get_applications(device)
    app_ids = []
    for application in sorted(applications, key=cmp_to_key(compare_applications)):
        app_ids.append(application.identifier)
    return app_ids


def load_js_file(session, filename):
    source = ''
    with codecs.open(filename, 'r', 'utf-8') as f:
        source = source + f.read()
    script = session.create_script(source)
    script.on('message', on_message)
    script.load()

    return script


def create_dir(path):
    path = path.strip()
    path = path.rstrip('\\')
    if os.path.exists(path):
        shutil.rmtree(path)
    try:
        os.makedirs(path)
    except os.error as err:
        print(err)


def open_target_app(device, name_or_bundleid):
    print('Start the target app {}'.format(name_or_bundleid))

    pid = ''
    session = None
    display_name = ''
    bundle_identifier = ''
    for application in get_applications(device):
        if name_or_bundleid == application.identifier or name_or_bundleid == application.name:
            pid = application.pid
            display_name = application.name
            bundle_identifier = application.identifier

    try:
        if not pid:
            pid = device.spawn([bundle_identifier])
            session = device.attach(pid)
            device.resume(pid)
        else:
            session = device.attach(pid)
    except Exception as e:
        print(e) 

    return session, display_name, bundle_identifier


def start_dump(session, ipa_name):
    print('Dumping {} to {}'.format(display_name, TEMP_DIR))

    script = load_js_file(session, DUMP_JS)
    script.post('dump')
    finished.wait()

    generate_ipa(PAYLOAD_PATH, ipa_name)

    if session:
        session.detach()

In [2]:
from __future__ import print_function
from __future__ import unicode_literals
import sys
import codecs
import frida
import threading
import os
import shutil
import time
import argparse
import tempfile
import subprocess
import re
import paramiko
from paramiko import SSHClient
from scp import SCPClient
from tqdm import tqdm
import traceback

IS_PY2 = sys.version_info[0] < 3
if IS_PY2:
    reload(sys)
    sys.setdefaultencoding('utf8')
    
DUMP_JS = './dump.js'

User = 'root'
Password = 'jj1216'
Host = 'localhost'
Port = 2222
KeyFileName = None

In [3]:
device = get_usb_iphone()

app_ids = []
if os.path.exists("./apps_list.txt"):
    with open("./apps_list.txt","r") as text_file:
        for line in text_file:
            app_ids.append(line.rstrip())

else:
    
    app_ids = get_application_IDs()
    for app_id in app_ids:

        with open("apps_list.txt","a") as text_file:
            text_file.write(app_id+"\n")

Waiting for USB device...


In [4]:
app_ids

['com.apple.mobilemail',
 'com.adobe.Adobe-Reader',
 'com.ZWQWGRSN4T.com.rileytestut.AltStore',
 'com.apple.AppStore',
 'com.apple.iBooks',
 'U com.dirtybeans.CSources2',
 'com.apple.calculator',
 'com.apple.mobilecal',
 'com.intsig.camcard.eng',
 'com.intsig.camcard',
 'com.intsig.CamScanner',
 'com.intsig.CamScannerPro',
 'com.apple.camera',
 'com.taptaptap.CameraPlus',
 'com.cardiio.cardiio.iphone',
 'com.google.chrome.ios',
 'com.apple.mobiletimer',
 'com.apple.compass',
 'com.apple.itunesconnect.mobile',
 'com.apple.MobileAddressBook',
 'com.google.GooglePlus',
 'com.saurik.Cydia',
 'com.digitimes.digitimes-news-adhoc',
 'com.dji.mimo',
 'com.hivebrain.DeepSleep',
 'com.reference.dictionary.dictionary',
 'com.getdropbox.Dropbox',
 'com.flir.consumer.flir.cloud',
 'com.flir.consumer.fx',
 'com.flir.one',
 'com.flir.tools',
 'com.apple.facetime',
 'com.facebook.Facebook',
 'com.apple.DocumentsApp',
 'com.apple.mobileme.fmf1',
 'com.apple.mobileme.fmip1',
 'com.google.Maps',
 'com.ap

In [5]:
import subprocess

In [7]:
for idx, app_id in enumerate(app_ids[100:104],100):
    print("idx= {}".format(idx))
    command = ['python' , 'dump.py' , '-u' , 'root' , '-P' , 'jj1216' , '-d' , "./ipas" , app_id]
    cmd = subprocess.Popen(command)
    cmd.communicate()

idx= 100
Start the target app cn.zhangyunbook.inovel
Dumping 掌雲書城 to /var/folders/42/14b610s56kqckbgzdq8gf5m80000gn/T
[frida-ios-dump]: Load Branch.framework success. 
[frida-ios-dump]: FirebaseCore.framework has been loaded. 
[frida-ios-dump]: Load FirebaseInstanceID.framework success. 
[frida-ios-dump]: Load FLAnimatedImage.framework success. 
[frida-ios-dump]: Load YYText.framework success. 
[frida-ios-dump]: Load TZImagePickerController.framework success. 
[frida-ios-dump]: Load OpenUDID.framework success. 
[frida-ios-dump]: Load HappyDNS.framework success. 
[frida-ios-dump]: Load FBLPromises.framework success. 
[frida-ios-dump]: Load FMDB.framework success. 
[frida-ios-dump]: Load CRBoxInputView.framework success. 
[frida-ios-dump]: Load SZTextView.framework success. 
[frida-ios-dump]: Load FirebaseInstallations.framework success. 
[frida-ios-dump]: Load AFNetworking.framework success. 
[frida-ios-dump]: Load Mantle.framework success. 
[frida-ios-dump]: Load FirebaseCoreDiagnostic

掌雲書城.fid: 100%|██████████| 19.3M/19.3M [00:07<00:00, 2.82MB/s]  
0.00B [00:00, ?B/s]

start dump /private/var/containers/Bundle/Application/A7D42304-5466-4809-A444-C046D00607AC/掌雲書城.app/Frameworks/AFNetworking.framework/AFNetworking
start dump /private/var/containers/Bundle/Application/A7D42304-5466-4809-A444-C046D00607AC/掌雲書城.app/Frameworks/Aspects.framework/Aspects


AFNetworking.fid: 100%|██████████| 532k/532k [00:00<00:00, 3.36MB/s]
Aspects.fid: 100%|██████████| 125k/125k [00:00<00:00, 886kB/s] 
0.00B [00:00, ?B/s]

start dump /private/var/containers/Bundle/Application/A7D42304-5466-4809-A444-C046D00607AC/掌雲書城.app/Frameworks/Branch.framework/Branch
start dump /private/var/containers/Bundle/Application/A7D42304-5466-4809-A444-C046D00607AC/掌雲書城.app/Frameworks/CRBoxInputView.framework/CRBoxInputView


...

start dump /private/var/containers/Bundle/Application/91BA1ECA-CC26-43B1-903E-0222564EFB61/QDReaderAppStore.app/Frameworks/libpag.framework/libpag


libpag.fid: 100%|██████████| 4.64M/4.64M [00:00<00:00, 21.9MB/s]
0.00B [00:00, ?B/s]

start dump /private/var/containers/Bundle/Application/91BA1ECA-CC26-43B1-903E-0222564EFB61/QDReaderAppStore.app/Frameworks/QDReaderExtensionShared.framework/QDReaderExtensionShared


QDReaderExtensionShared.fid: 100%|██████████| 314k/314k [00:00<00:00, 2.54MB/s]
qidian_logo.svg: 165MB [00:18, 9.36MB/s]                                  
0.00B [00:00, ?B/s]


Generating "起点读书 -- m.qidian.QDReaderAppStore.ipa"


In [ ]:
app_id = 'com.tencent.mqq'
command = ['python' , 'dump.py' , '-u' , 'root' , '-P' , 'jj1216' , '-d' , "./ipas" , app_id]
cmd = subprocess.Popen(command)
cmd.communicate()